In [1]:

import tensorflow as tf
print('TF Version: ', tf.__version__)
from platform import python_version
print('Python Version: ', python_version())

import os
import shutil

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

2023-05-19 09:57:28.268819: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 09:57:28.300908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 09:57:28.796200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TF Version:  2.12.0
Python Version:  3.10.6


In [2]:
GPU = 3 # define the GPU to use
# Set the GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU)
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [3]:


"""
## Setup
"""


import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from sklearn.metrics import calinski_harabasz_score

tfds.disable_progress_bar()
autotune = tf.data.AUTOTUNE


"""
## Prepare the dataset

"""

# Load the horse-zebra dataset using tensorflow-datasets.
#dataset, _ = tfds.load("cycle_gan/horse2zebra", with_info=True, as_supervised=True)
noisy_tr = np.load('hn_train_set.npy')
clean_tr = np.load('ln_train_set.npy')
noisy_va = np.load('hn_valid_set.npy')
clean_va = np.load('ln_valid_set.npy')

noisy_va_sup = np.load('hn_valid_set_sup.npy')
clean_va_sup = np.load('ln_valid_set_sup.npy')

noisy_te = np.load('hn_test_set.npy')
clean_te = np.load('ln_test_set.npy')
#train_noisy, train_clean = np.expand_dims(noisy_tr[:200],axis=0), np.expand_dims(clean_tr[:200],axis=0)
#test_noisy, test_clean = np.expand_dims(noisy_tr[:1],axis=0), np.expand_dims(clean_tr[:1],axis=0)
train_noisy, train_clean = noisy_tr, clean_tr
valid_noisy, valid_clean = noisy_va, clean_va
valid_noisy_sup, valid_clean_sup = noisy_va_sup, clean_va_sup
test_noisy, test_clean = noisy_te, clean_te



input_spec_size = (500,1)

buffer_size = 256
batch_size = 5




def get_AE(
    name=None,
):
    spec_input = layers.Input(shape=input_spec_size, name=name + "_spec_input")
    x1 = layers.ZeroPadding1D(padding=6)(spec_input)
    x1 = layers.Conv1D(64,7, activation='relu', padding='same')(x1)
    #x1 = layers.Conv1D(32,3, activation='relu', padding='same')(x1)
    x2 = layers.MaxPooling1D(2)(x1)
    x3 = layers.Conv1D(32,7, activation='relu', padding='same')(x2)
    #x3 = layers.Conv1D(32,3, activation='relu', padding='same')(x3)
    x4 = layers.MaxPooling1D(2)(x3)
    x5 = layers.Conv1D(16,7, activation='relu', padding='same')(x4)
    #x5 = layers.Conv1D(32,3, activation='relu', padding='same')(x5)
    x6 = layers.MaxPooling1D(2)(x5)
    x7 = layers.Conv1D(8,7, activation='relu', padding='same')(x6)
    x7 = layers.MaxPooling1D(2)(x7)
    x7 = layers.Conv1D(1,7, activation='relu', padding='same')(x7)
    #x7 = layers.Conv1D(16,3, activation='relu', padding='same')(x7)
    d4 = layers.UpSampling1D(2)(x7)
    d5 = layers.Conv1D(8,7,strides=1, activation='relu', padding='same')(d4)
    d5 = layers.UpSampling1D(2)(d5)
    d5 = layers.Conv1D(16,7,strides=1, activation='relu', padding='same')(d5)
    #d5 = layers.Conv1D(32,1,strides=1, activation='relu', padding='same')(d5)
    d6 = layers.UpSampling1D(2)(d5)
    d7 = layers.Conv1D(32,7,strides=1, activation='relu', padding='same')(d6)
    #d7 = layers.Conv1D(32,1,strides=1, activation='relu', padding='same')(d7)
    d8 = layers.UpSampling1D(2)(d7)
    d9 = layers.Conv1D(64,7,strides=1, activation='relu', padding='same')(d8)
    #d9 = layers.Conv1D(32,1,strides=1, activation='relu', padding='same')(d9)
    d9 = layers.Cropping1D(6)(d9)
    decoded = layers.Conv1D(1,1,strides=1, padding='same')(d9)
    decoded = layers.LeakyReLU(alpha=0.3)(decoded)
    model = keras.models.Model(spec_input, decoded, name=name)
    model.summary()
    return model



# Get the model
net_AE = get_AE(name="AE")

"""
## Build the model

We will override the `train_step()` method of the `Model` class
for training via `fit()`.
"""


class AE(keras.Model):
    def __init__(
        self,
        network_AE,
    ):
        super().__init__()
        self.net_AE = network_AE

    def call(self, inputs):
        return (
            self.net_AE(inputs),

        )

    def compile(
        self,
        net_AE_optimizer,
        AE_loss_fn,

    ):
        super().compile()
        self.net_AE_optimizer = net_AE_optimizer
        self.AE_loss_fn = AE_loss_fn

    def train_step(self, batch_data):
        # x is noisy and y is high SNR
        spec_x, spec_y = batch_data

        with tf.GradientTape(persistent=True) as tape:
            # denoise spectra
            denoised_y = self.net_AE(spec_x, training=True)
            
            # AE adverserial loss
            net_AE_loss = self.AE_loss_fn(denoised_y,spec_y)
            
            # Total AE loss
            total_loss_AE = net_AE_loss 



        # Get the gradients for the model
        grads_AE = tape.gradient(total_loss_AE, self.net_AE.trainable_variables)


        # Update the weights of the model
        self.net_AE_optimizer.apply_gradients(
            zip(grads_AE, self.net_AE.trainable_variables)
        )

        return {
            "AE_loss": total_loss_AE,
        }


"""
## Create a callback that periodically saves spectra
"""


class GANMonitor(keras.callbacks.Callback):
    """A callback to generate and save spectra after each epoch"""
    
    def on_epoch_end(self, epoch, logs=None):
        #manually batch test set (200), and evaluate them
        #save noisy network inputs, denoised spectra
        #and ground truths

        spectra = test_noisy
        prediction = np.zeros(np.shape(spectra))
        GTS = np.zeros(np.shape(spectra))
        inputs = np.zeros(np.shape(spectra))
        counter = 0
        for i in range(200, np.shape(spectra)[0], 200):
            prediction[i-200:i,:] = np.squeeze(self.model.net_AE(spectra[i-200:i]))
            GTS[i-200:i,:] = test_clean[i-200:i]
            inputs[i-200:i,:] = spectra[i-200:i]
            counter = counter+1
        # get remaining bit of last batch
        prediction[(200*counter):] = np.squeeze(self.model.net_AE(spectra[(200*counter):]))
        GTS[(200*counter):] = test_clean[(200*counter):]
        inputs[(200*counter):] = spectra[(200*counter):]
        
        prediction = np.reshape(prediction,(-1,500))
        GTS = np.reshape(GTS,(-1,500))
        inputs = np.reshape(inputs,(-1,500))
        path = './epoch_' + str(epoch)
        if os.path.exists(path):
            shutil.rmtree(path)
        os.mkdir(path)
        np.save(path + '/AE_denoised', prediction)
        np.save(path + '/AE_denoised_GT', GTS)
        np.save(path + '/AE_input', inputs)

        
        
        # compute unsupervised validation loss
        spectra_valid = valid_noisy
        prediction_valid = np.zeros(np.shape(spectra_valid))
        GTS_valid = np.zeros(np.shape(spectra_valid))
        inputs_valid = np.zeros(np.shape(spectra_valid))
        
        counter = 0
        for i in range(200, np.shape(spectra_valid)[0], 200):
            prediction_valid[i-200:i,:] = np.squeeze(self.model.net_AE(spectra_valid[i-200:i]))
            GTS_valid[i-200:i,:] = valid_clean[i-200:i]
            inputs_valid[i-200:i,:] = spectra_valid[i-200:i]
            counter = counter+1
        # get remaining bit of last batch
        prediction_valid[(200*counter):] = np.squeeze(self.model.net_AE(spectra_valid[(200*counter):]))
        GTS_valid[(200*counter):] = valid_clean[(200*counter):]
        inputs_valid[(200*counter):] = spectra_valid[(200*counter):]
        
        prediction_valid = np.reshape(prediction_valid,(-1,500))
        GTS_valid = np.reshape(GTS_valid,(-1,500))
        inputs_valid = np.reshape(inputs_valid,(-1,500))
        
        cluster_true = KMeans(8, random_state=4).fit(GTS_valid)
        cluster_pred = cluster_true.predict(prediction_valid)
        print(np.shape(cluster_pred))
        
        cluster_true = cluster_true.labels_
        #cluster_pred = cluster_pred.labels_
        valid_loss = metrics.calinski_harabasz_score(prediction_valid, cluster_pred)
        valid_loss_rand = adjusted_rand_score(cluster_true,cluster_pred)
        np.save(path + '/AE_valid_loss_' + str(epoch), valid_loss)
        np.save(path + '/AE_valid_loss_rand_' + str(epoch), valid_loss_rand)
        
        # compute supervised validation loss
        spectra_valid_sup = valid_noisy_sup
        prediction_valid_sup = np.zeros(np.shape(spectra_valid_sup))
        GTS_valid_sup = np.zeros(np.shape(spectra_valid_sup))
        inputs_valid_sup = np.zeros(np.shape(spectra_valid_sup))
        
        counter = 0
        for i in range(200, np.shape(spectra_valid_sup)[0], 200):
            prediction_valid_sup[i-200:i,:] = np.squeeze(self.model.net_AE(spectra_valid_sup[i-200:i]))
            GTS_valid_sup[i-200:i,:] = valid_clean_sup[i-200:i]
            inputs_valid_sup[i-200:i,:] = spectra_valid_sup[i-200:i]
            counter = counter+1
        # get remaining bit of last batch
        prediction_valid_sup[(200*counter):] = np.squeeze(self.model.net_AE(spectra_valid_sup[(200*counter):]))
        GTS_valid_sup[(200*counter):] = valid_clean_sup[(200*counter):]
        inputs_valid_sup[(200*counter):] = spectra_valid_sup[(200*counter):]
        
        prediction_valid_sup = np.reshape(prediction_valid_sup,(-1,500))
        GTS_valid_sup = np.reshape(GTS_valid_sup,(-1,500))
        inputs_valid_sup = np.reshape(inputs_valid_sup,(-1,500))
        
        valid_loss_sup = np.mean(np.mean((np.squeeze(prediction_valid_sup) - np.squeeze(GTS_valid_sup))**2,axis=1))
        valid_loss_trad_AE = np.mean(np.mean((np.squeeze(prediction_valid_sup) - np.squeeze(inputs_valid_sup))**2,axis=1))
        np.save(path + '/AE_valid_loss_sup_' + str(epoch), valid_loss_sup)
        np.save(path + '/valid_loss_trad_AE_' + str(epoch), valid_loss_trad_AE)

"""
## Train the end-to-end model
"""


# Loss function for evaluating adversarial loss
adv_loss_fn = keras.losses.MeanSquaredError()

# Define the loss function for the generators


def AE_loss_fn(fake,real):
    fake_loss = adv_loss_fn(fake, real)
    return fake_loss



# Create cycle gan model
AE_model = AE(
    network_AE=net_AE
)

# Compile the model
AE_model.compile(
    net_AE_optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-4, beta_1=0.5),
    AE_loss_fn=AE_loss_fn,
)
# Callbacks
plotter = GANMonitor()
checkpoint_filepath = "./model_checkpoints/cyclegan_checkpoints.{epoch:03d}"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, save_weights_only=True
)



/home/ciaran/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/ciaran/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-14 15:47:56.171520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22071 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus

In [ ]:
train_noisy = tf.data.Dataset.from_tensor_slices((train_noisy))
train_clean = tf.data.Dataset.from_tensor_slices((train_clean))
#test_horses = tf.data.Dataset.from_tensor_slices((test_horses))
#test_zebras = tf.data.Dataset.from_tensor_slices((test_zebras))
train_noisy = train_noisy.batch(batch_size)
train_clean = train_clean.batch(batch_size)



history = AE_model.fit(
    tf.data.Dataset.zip((train_noisy, train_noisy)),
    epochs=100,
    callbacks=[plotter, model_checkpoint_callback],
)

np.save('my_history.npy',history.history)

2023-05-14 15:47:57.953546: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [68925,500]
	 [[{{node Placeholder/_5}}]]
2023-05-14 15:47:57.953646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [68925,500]
	 [[{{node Placeholder/_5}}]]
2023-05-14 15:48:19.047940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


 7530/13785 [===============>..............] - ETA: 5:26 - G_loss: 0.3254 - F_loss: 0.4121 - D_X_loss: 0.2510 - D_Y_loss: 0.2450

In [ ]:


#history=np.load('my_history.npy',allow_pickle='TRUE').item()



# Load the checkpoints
weight_file = "./saved_checkpoints/cyclegan_checkpoints.001"
cycle_gan_model.load_weights(weight_file).expect_partial()
print("Weights loaded successfully")

